# Run this cell first

In [1]:
# this code enables the automated feedback. If you remove this, you won't get any feedback
# so don't delete this cell!
try:
  import AutoFeedback
except (ModuleNotFoundError, ImportError):
  %pip install AutoFeedback
  import AutoFeedback

try:
  from testsrc import test_main
except (ModuleNotFoundError, ImportError):
  %pip install "git+https://github.com/autofeedback-exercises/exercises.git#subdirectory=MTH2031/crashcourse/inertia_tensor"
  from testsrc import test_main

def runtest(tlist):
  import unittest
  from contextlib import redirect_stderr
  from os import devnull
  with redirect_stderr(open(devnull, 'w')):
    suite = unittest.TestSuite()
    for tname in tlist:
      suite.addTest(eval(f"test_main.UnitTests.{tname}"))
    runner = unittest.TextTestRunner()
    try:
      runner.run(suite)
    except AssertionError:
      pass

# Application of Eigenvectors: Inertia Tensor

One application of eigenvectors that we will see in this module is computing, from the inertia tensor, the principle moments of inertia of a system. In the example below, you will set up a collection of $N$ particles of equal mass, randomly distributed within a space, set up the inertia tensor, and plot the particles along with the principle moments of inertia.

# TASKS

We will work with $N=100$ particles. `N` is already defined. In the cell below set up arrays for 

1. `m`: the mass of each particle. To begin with, we'll just make all the masses equal to 1.
2. `x`: the $x$ coordinate of each particle. These should be normally randomly distributed about zero with a standard deviation of 1. You can use [`np.random.normal`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html) for this.
3. `y`: the $y$ coordinate of each particle, normally randomly distributed about zero with a standard deviation of 2.
4. `z`: the $z$ coordinate of each particle, normally randomly distributed about zero with a standard deviation of 3.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

N = 100

m = np.ones(N)
x = np.random.normal(0, 1, N)
y = np.random.normal(0, 2, N)
z = np.random.normal(0, 3, N)

In [ ]:
runtest(['test_vars'])

## Centre of mass

The total mass is just the sum of the mass of all the particles. 

$$M = \sum _i m_n$$

To calculate the centre of mass we need

$$X_c = \sum _i ^N \dfrac{m_i x_i}{M}, $$

and the equivalent for the $Y_c$ and $Z_c$ coordinate.

Then to shift all of the coordinates so that they are centred on the centre of mass, simply subtract the centre of mass from each coordinate:

$$x'_i = x_i - X_c.$$

Define the variables

1. `M`: the total mass of the particles in the system
2. `Xc`, `Yc`, `Zc`: the coordinates of the centre of mass of the system
3. `x1`, `y1`, `z1`: the coordinates of the particles, relative to the centre of mass.

In [4]:
M = np.sum(m)

Xc = np.sum(m*x)/M
Yc = np.sum(m*y)/M
Zc = np.sum(m*z)/M

x1 = x - Xc
y1 = y - Yc
z1 = z - Zc

In [ ]:
runtest(['test_vars_2'])


Recall that the inertia tensor is then given by

$$
\mathbf{I} = 
\left( \begin{align*}
  I_{xx} & - I_{xy} & - I_{xz} \\
- I_{yx} & \phantom{-}  I_{yy} & - I_{yz} \\
- I_{zx} & - I_{zy} &   I_{zz}
\end{align*}
\right)
$$

where

$$ I_{xx} = \int _m (y'^2+z'^2) dm, \quad
   I_{yy} = \int _m (x'^2+z'^2) dm, \quad
   I_{zz} = \int _m (x'^2+y'^2) dm
$$

and 

$$ I_{xy} = I_{yz} = \int _m x'y' dm, \quad
   I_{xz} = I_{xz} = \int _m x'z' dm, \quad
   I_{yz} = I_{xz} = \int _m y'z' dm
$$

To approximate these integrals, we will compute them numerically as the sum over the particles, for instance:

$$ I_{xx} \approx \sum \limits _{i} ^{N} m_i ({y'_i}^2 + {z'_i}^2) $$

Set up the inertia tensor, `I` by computing the nine values. The first one is done for you so that you can see the syntax.

In [ ]:
I = np.zeros((3,3))

I[0,0] = np.sum( m * (y1**2 + z1**2))
I[1,1] = np.sum( m * (x1**2 + z1**2))
I[2,2] = np.sum( m * (x1**2 + y1**2))

I[0,1] = -np.sum( m * x1 * y1)
I[0,2] = -np.sum( m * x1 * z1)
I[1,2] = -np.sum( m * y1 * z1)

I[1,0] = I[0,1]
I[2,0] = I[0,2]
I[2,1] = I[1,2]

print('Inertia tensor\n',I)

In [ ]:
runtest(['test_tensor'])

---

The code for the rest of this demo is already completed for you, as it uses syntax and commands you won't have seen before. If you have set everything up correctly, then you should be able to execute the code cells below to produce a plot of the system. 

Now, to compute the principal moments and axes of inertia, we simply compute the eigenvalues and eigenvectors of the inertia tensor. 

In [ ]:
Ipr, Vpr = np.linalg.eig(I)
print('Principal moments of inertia\n',Ipr)
print()
print('Principal axes of inertia\n',Vpr)

Finally, we produce a plot of the system. The individual plot commands are explained in the commented code below.

In [ ]:
ax = plt.axes(projection='3d')
"""
Plot the individual particles with a magenta point (m.)
Plot the centre of mass as a red point (r.)
"""
ax.plot3D(x, y, z, 'm.', markersize=10)
ax.plot3D(Xc, Yc, Zc, 'r.')

"""set the axes labels and aspect ratio"""
ax.set_aspect('equal')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

"""
To plot the axes of inertia, we will plot arrows whose length is scaled by the corresponding moment of inertia. The scaling factors have the dimension length proportional to the square root of the principal moments of inertia
"""

S1 = np.sqrt(Ipr[0]/M)
S2 = np.sqrt(Ipr[1]/M)
S3 = np.sqrt(Ipr[2]/M)

"""
We now use the 'quiver' package to plot the arrows. Each arrow starts at the centre of mass (Xc, Yc, Zc), and terminates at the coordinates given by the (scaled) values of the principal axes. Thus the first arrow, plotted in red terminates at coordinates S1(Vpr[0, 0], Vpr[1, 0], Vpr[2, 0]). 
"""
ax.quiver(Xc,Yc,Zc,S1*Vpr[0,0],S1*Vpr[1,0],S1*Vpr[2,0], color='r', arrow_length_ratio = 0.3)
ax.quiver(Xc,Yc,Zc,S2*Vpr[0,1],S2*Vpr[1,1],S2*Vpr[2,1], color='g', arrow_length_ratio = 0.3)
ax.quiver(Xc,Yc,Zc,S3*Vpr[0,2],S3*Vpr[1,2],S3*Vpr[2,2], color='b', arrow_length_ratio = 0.3)

plt.show()

# Taking it further

Once you have completed the code, passed the tests and produced the plot, you might try editing to code to describe a different system. What happens for instance if the masses of the particles are not all equal? What about the following problems?

To make it easier (and to avoid copying a bunch of code), I've defined a function, plot_system for you which takes as its arguments, the lists of masses, $x$, $y$ and $z$ coordinates, and produces the plot. You can call it like this. (Note, this will only work if you have defined m, x, y and z already in the code cells above)

In [ ]:
from testsrc.test_main import plot_system

plot_system(m, x, y, z)

In [ ]:
# Try setting up the arrays m, x, y and z to assign a different mass to each of the randomly distributed particles. 



---

As well as trying changing the masses, why not have a go at these:




1. The propane molecule is composed of three (heavy) carbon atoms, and 8 (light) hydrogens like so
<center><img src='https://raw.githubusercontent.com/autofeedback-exercises/exercises/main/MTH2031/crashcourse/inertia_tensor/propane.jpg' height='100'/></center>
![propane](./propane.jpg)

We can set the coordinates for the atoms as follows:

In [ ]:
x, y, z = np.zeros(11)

x[0], y[0], z[0] = [3.3709, 1.6468, 1.3253]
x[1], y[1], z[1] = [2.0046, 1.0757, 0.9328]
x[2], y[2], z[2] = [0.9486, 1.2446, 2.0298]
x[3], y[3], z[3] = [3.7626, 1.1536, 2.2237]
x[4], y[4], z[4] = [3.3033, 2.7202, 1.5421]
x[5], y[5], z[5] = [4.1075, 1.5143, 0.5244]
x[6], y[6], z[6] = [1.6559, 1.5639, 0.0126]
x[7], y[7], z[7] = [2.1109, 0.0097, 0.6897]
x[8], y[8], z[8] = [-0.0189, 0.8295, 1.7244]
x[9], y[9], z[9] = [0.7958, 2.3038, 2.2719]
x[10], y[10], z[10] = [1.2529, 0.7365, 2.9535]

where the first three values correspond to the three carbon atoms. The C atoms have a mass of 12 (atomic mass units) and the hydrogens, 1 amu. Set up the list of masses accordingly, and then you can use the `plot_system` command as before.

---

2. Instead of distributing the particles randomly, what happens if they are uniformly distributed within a particular volume, for instance a rectangular parallelepiped? 
   
(HINT: decide the dimensions of your paralleleped first (say $a\times b \times c)$, then place a particle of unit mass at each integer grid point within the volume of the parallelepiped. That will tell you how many particles you need (i.e. how big are the arrays), and you can set up loops which go through each point and set the `x`, `y` and `z` coordinates appropriately.)